In [32]:
import redis
import json

class RedisChannel:
    do_tts_service = "do-tts-service"
    tts_done_service = "tts-done-service"
    do_asr_service = "do-asr-service"
    asr_done_service = "asr-done-service"
    
def publisher(channel, message):
    redis_client_pub.publish(channel, message)
    print(f"Published: {message}")
redis_client_pub = redis.Redis(host='localhost', port=51201)

# # publisher(RedisChannel.do_asr_service, 3333)

message = json.dumps({"state":1})
publisher(RedisChannel.do_asr_service, message)

Published: {"state": 1}


In [ ]:
import os
import sys

project_path = os.getcwd()
print(project_path)
sys.path.append(f'{project_path}/packages')
from WhisperLive.whisper_live.client import TranscriptionClient
# from whisper_live.client import TranscriptionClient
client = TranscriptionClient(
  "localhost",
  9090,
  lang="zh",
  translate=False,
  model="small",
  use_vad=True,
)
client()

In [ ]:
# -*- coding: utf-8 -*-


from vits.text import text_to_sequence
import vits.commons as commons
import vits.utils as utils
from vits.models import SynthesizerTrn
from torch import no_grad, LongTensor
import torch

import wave
import numpy as np
from datetime import datetime

class VitsService:
    def __init__(self,hparams_file_path = "./models/community/config.json",checkpoint_path="./models/community/G_latest.pth"):
        self._hparams_file_path = hparams_file_path 
        self._checkpoint_path = checkpoint_path
        self.device = "cuda:0" if torch.cuda.is_available() else "cpu"
        self.language_marks = {
            "Japanese": "",
            "日本語": "[JA]",
            "中文": "[ZH]",
            "English": "[EN]",
            "Mix": "",
            }
        self.lang = ['日本語', '中文', 'English', 'Mix']
        self.hps = utils.get_hparams_from_file(self._hparams_file_path)
        self.set_model()

    def set_model(self):
        self.net_g = SynthesizerTrn(
            len(self.hps.symbols),
            self.hps.data.filter_length // 2 + 1,
            self.hps.train.segment_size // self.hps.data.hop_length,
            n_speakers=self.hps.data.n_speakers,
            **self.hps.model).to(self.device)
        _ = self.net_g.eval()

        _ = utils.load_checkpoint(self._checkpoint_path, self.net_g, None)

        self.speaker_ids = self.hps.speakers

    def get_text(self,text, hps, is_symbol):
        text_norm = text_to_sequence(text, hps.symbols, [] if is_symbol else hps.data.text_cleaners)
        if hps.data.add_blank:
            text_norm = commons.intersperse(text_norm, 0)
        text_norm = LongTensor(text_norm)
        return text_norm

    def create_tts_fn(self):
        def tts_fn(text, speaker, language, speed):
            if language is not None:
                text = self.language_marks[language] + text + self.language_marks[language]
            speaker_id = self.speaker_ids[speaker]
            stn_tst = self.get_text(text, self.hps, False)
            with no_grad():
                x_tst = stn_tst.unsqueeze(0).to(self.device)
                x_tst_lengths = LongTensor([stn_tst.size(0)]).to(self.device)
                sid = LongTensor([speaker_id]).to(self.device)
                audio = self.net_g.infer(x_tst, x_tst_lengths, sid=sid, noise_scale=.667, noise_scale_w=0.8,
                                    length_scale=1.0 / speed)[0][0, 0].data.cpu().float().numpy()
            del stn_tst, x_tst, x_tst_lengths, sid
            return "Success", (self.hps.data.sampling_rate, audio)

        return tts_fn


In [ ]:
import os
import sys

project_path = os.getcwd()
print(project_path)
sys.path.append(f'{project_path}/packages')

In [ ]:
import argparse
from typing import Optional
from WhisperLive.whisper_live.client import TranscriptionClient

import time
import threading

import rx
from rx import operators as ops

from opencc import OpenCC

# 创建转换器，从繁体转为简体
cc = OpenCC('t2s')  # t2s 表示 Traditional Chinese to Simplified Chinese

stop_event = threading.Event()
history_segments = []

class ResettableTimer:
    def __init__(self, interval, action):
        """
        Initialize the timer.
        初始化計時器。
        :param interval: the time interval in seconds.
                         設定的時間間隔（秒）。
        :param action: the function to execute after timeout.
                       超時後執行的動作（函數）。
        """
        self.interval = interval
        self.action = action
        self.timer = None
        self.lock = threading.Lock()

    def reset(self):
        """
        Reset the timer, to be called upon receiving a new event.
        重置計時器，每次接收到新的事件時調用。
        """
        with self.lock:
            if self.timer is not None:
                self.timer.cancel()
            self.timer = threading.Timer(self.interval, self.action)
            self.timer.start()

    def stop(self):
        """
        Stop the timer, to be called when no longer needed.
        停止計時器，當不再需要時調用。
        """
        with self.lock:
            if self.timer is not None:
                self.timer.cancel()
                self.timer = None
def optimize_segments(history_segments):
    """
    Optimize the list of segments by keeping only the segment with the maximum 'end' value for each 'start' time.
    優化片段列表，為每個起始時間只保留最大結束值的片段。

    :param history_segments: List of dictionaries, each containing 'start', 'end', and 'text' keys.
    :return: A list of optimized segments.
    """
    max_segments = {}

    for segment in history_segments:
        start = segment['start']
        end = float(segment['end'])
        text = segment['text']

        if start in max_segments:
            # Compare and keep the entry with the maximum 'end'
            # 比較並保留擁有最大結束時間的條目
            if end > float(max_segments[start]['end']):
                max_segments[start] = {'end': str(end), 'text': text}
        else:
            # Add new entry
            # 添加新條目
            max_segments[start] = {'end': str(end), 'text': text}

    # Convert the dictionary back to a list for the output
    # 將字典轉換回列表以輸出
    optimized_segments = [{'start': start, 'end': info['end'], 'text': info['text']} for start, info in max_segments.items()]
    return optimized_segments

def format_sentence(input_string):
    """

    Args:
        input_string (_type_): _description_

    Returns:
        _type_: _description_
    """
    # Replace spaces and commas with Chinese commas
    # 將空格和逗號替換為中文逗號
    formatted_string = input_string.replace(" ", "，").replace(",", "，")
    
    # Check if the sentence ends with a period, add one if not
    # 檢查句子是否以句號結尾，如果沒有則添加
    if not formatted_string.endswith("。"):
        formatted_string += "。"
    
    return formatted_string

resettable_timer_seconds = 3
def timeout_event():
    """
    Define the event to execute on timeout
    定義超時時執行的事件
    """
    print("= = = = END = = =\n\n")
    print(f"Timeout event triggered after {resettable_timer_seconds} seconds with no events!")
    print(f'format_sentence(segment_behavior_subject.value): {format_sentence(client.client.segment_behavior_subject.value)}')
    print(f'optimize_segments(history_segments): {optimize_segments(history_segments)}')
    
    stop_client()

# Define a global event for signaling the main thread that the thread has completed
# 定義一個全局事件以向主線程信號表示線程已完成
finished_event = threading.Event()

# Setup the TranscriptionClient
# 設置轉錄客戶端
client: Optional[TranscriptionClient] = None
resettable_timer = ResettableTimer(resettable_timer_seconds, timeout_event)  # Set 3-second timeout
# 設定3秒的超時

# Define a subscriber
# 定義一個訂閱者
def last_segment_handler(last_segment):
    simplified_text = cc.convert(last_segment['text'])
    last_segment['text'] = simplified_text
    print(f"[Received] last_segment_subject : {last_segment['text']}")
    history_segments.append(last_segment)

def segment_handler(segment):
    # simplified_text = cc.convert(segment)
    print(f"[Received] segment_subject : {segment}")
    resettable_timer.reset()  # Reset the timer on event reception
    # 接收到事件時重設計時器


# Function to start the TranscriptionClient
# 啟動轉錄客戶端的函數
def run_client():
    try:
        if client is not None:
            client()  # Assuming client() is blocking
    except Exception as e:
        print(f"Error during client run: {e}")
    finally:
        finished_event.set()

# Function to stop the WebSocket and TranscriptionClient
# 停止WebSocket和轉錄客戶端的函數
def stop_client():
    try:
        print("Stopping client...")
        client.client.close_websocket()
        resettable_timer.stop()  # Ensure timer is stopped
        # 確保計時器停止
        client_thread.join(timeout=30)  # Wait up to 30 seconds
        # 等待最多30秒
    except Exception as e:
        print(f"Error stopping client: {e}")
    finally:
        finished_event.set()  # Mark thread as completed when stopping
        # 停止時標記線程為已完成

def main():
    global client
    if client is None:
        client = TranscriptionClient(
            "localhost",
            port=9090,
            lang="zh",
            translate=False,
            model="small",
            use_vad=True,
        )
    # Subscribe to data
    # 訂閱數據
    last_segment_subscription = client.client.last_segment_behavior_subject.pipe(
        ops.filter(lambda last_segment: last_segment['text'] != ''),
        ops.distinct_until_changed()
    ).subscribe(last_segment_handler)

    segment_subscription = client.client.segment_behavior_subject.pipe(
        ops.filter(lambda segment: segment != ''),
        ops.distinct_until_changed()
    ).subscribe(segment_handler)
    
    client_thread = threading.Thread(target=run_client)
    client_thread.start()
    client_thread.join(timeout=60)  # Wait up to 60 seconds
    # 等待最多60秒
    if client_thread.is_alive():
        stop_client()

# args:Optional[argparse.Namespace ] = None
# Execute the main function when running the script
# 執行腳本時運行主函數
if __name__ == "__main__":
    main()


In [ ]:
import asyncio
from whisper_live.client import TranscriptionClient
import time
import threading

import rx
from rx import operators as ops
from rx.scheduler.eventloop import AsyncIOScheduler

# 建立一個非同步的重設計時器
class AsyncResettableTimer:
    def __init__(self, interval, action):
        self.interval = interval
        self.action = action
        self.task = None

    async def reset(self):
        if self.task is not None:
            self.task.cancel()
        self.task = asyncio.create_task(self.run())

    async def run(self):
        await asyncio.sleep(self.interval)
        self.action()

    async def stop(self):
        if self.task is not None:
            self.task.cancel()
            self.task = None

# 超時事件函數
def timeout_event():
    print("三秒內沒有收到事件，觸發了超時事件！")

client = TranscriptionClient(
        "localhost",
        9090,
        lang="zh",
        translate=False,
        model="small",
        use_vad=True,
    )
async_resettable_timer = AsyncResettableTimer(3, timeout_event)

async def print_item(item):
    print(f"RX Received: {item}")
    await async_resettable_timer.reset()  # 收到事件，重置計時器



# 訂閱數據
# subscription = client.client.segments_sub.pipe(
#     ops.distinct_until_changed()
# ).subscribe(print_item)

# 定義一個非同步的 TranscriptionClient 使用函數
async def run_client(client, timer):
    try:
        await client()  # 假設這個函數是非同步的
        await async_resettable_timer.reset()
    except Exception as e:
        print(f"Error during client run: {e}")
    finally:
        finished_event.set()

async def main():# 創建一個 AsyncIO 調度器
    loop = asyncio.get_event_loop()
    scheduler = AsyncIOScheduler(loop)

    # 使用調度器來訂閱
    subscription = client.client.segments_sub.pipe(
        ops.distinct_until_changed()
    ).subscribe(print_item, scheduler=scheduler)

    await run_client(client, async_resettable_timer)
    # 等待一個模擬的長時間操作，比如說運行客戶端
    await asyncio.sleep(30)  # 假設客戶端運行 30 秒
    await async_resettable_timer.stop()

# 在 Jupyter Notebook 中啟動非同步事件循環
await main()


In [ ]:
from fastapi import FastAPI, WebSocket, WebSocketDisconnect, HTTPException
from pydantic import BaseModel
from typing import List

app = FastAPI()

class Item(BaseModel):
    name: str
    description: str | None = None
    price: float
    tax: float | None = None

items_db: List[Item] = [
    {"name":"Jones","price":30}
]

@app.get("/")
def read_root():
    return {"message": "Welcome to FastAPI!"}

@app.get("/items/", response_model=List[Item])
def read_items():
    return items_db

@app.get("/items/{item_id}", response_model=Item)
def read_item(item_id: int):
    if 0 <= item_id < len(items_db):
        return items_db[item_id]
    raise HTTPException(status_code=404, detail="Item not found")

@app.post("/items/", response_model=Item)
def create_item(item: Item):
    items_db.append(item)
    return item

# WebSocket 管理器
class ConnectionManager:
    def __init__(self):
        self.active_connections: List[WebSocket] = []

    async def connect(self, websocket: WebSocket):
        await websocket.accept()
        self.active_connections.append(websocket)

    def disconnect(self, websocket: WebSocket):
        self.active_connections.remove(websocket)

    async def send_personal_message(self, message: str, websocket: WebSocket):
        await websocket.send_text(message)

    async def broadcast(self, message: str):
        for connection in self.active_connections:
            await connection.send_text(message)

manager = ConnectionManager()

@app.websocket("/ws")
async def websocket_endpoint(websocket: WebSocket):
    await manager.connect(websocket)
    try:
        while True:
            data = await websocket.receive_text()
            await manager.broadcast(f"Client said: {data}")
    except WebSocketDisconnect:
        manager.disconnect(websocket)
        await manager.broadcast("Client disconnected")

In [ ]:

import nest_asyncio
import uvicorn
import threading

# 使用 nest_asyncio 允許在 Jupyter Notebook 中運行異步應用
nest_asyncio.apply()

# 使用 uvicorn 運行應用
config = uvicorn.Config(app, host="localhost", port=8000, log_level="info")
server = uvicorn.Server(config)

def run_server():
    server.run()


# 啟動 TranscriptionClient 在一個新的執行緒中
server_thread = threading.Thread(target=run_server, daemon=True)
server_thread.start()

# 模擬客戶端運行一段時間後停止
async def stop_server():
    # 假設已有全局 server 實例
    server.should_exit = True  # 通知 uvicorn 伺服器需要退出
    await server.wait_closed()  # 等待伺服器關閉
    stop_event.set()
    server_thread.join()
    print("WebSocket 伺服器已停止")

In [ ]:
import asyncio
import websockets
import threading
import time


async def websocket_client():
    uri = "ws://localhost:8000/ws"
    async with websockets.connect(uri) as websocket:
        await websocket.send("Hello, WebSocket!")
        while not stop_event.is_set():
            try:
                message = await asyncio.wait_for(websocket.recv(), timeout=1.0)
                print(f"Received: {message}")
            except asyncio.TimeoutError:
                continue
            except websockets.exceptions.ConnectionClosed:
                break  # 連線被關閉時跳出循環

def run_websocket_client():
    asyncio.new_event_loop().run_until_complete(websocket_client())

# 啟動 WebSocket 客戶端
websocket_client_thread = threading.Thread(target=run_websocket_client, daemon=True)
websocket_client_thread.start()

# 模擬客戶端運行一段時間後停止
def stop_websocket_client():
    stop_event.set()
    websocket_client_thread.join()
    print("WebSocket 客戶端已停止")

# # 運行 10 秒後停止
# time.sleep(10)
# stop_websocket_client()
# print("WebSocket 客戶端已停止")


In [ ]:
stop_server()
stop_websocket_client()

